# Deep Network for Design Parameters for FWUAV

Importing the Modules

In [1]:
# Modules to import for data preprocessing
import numpy as np
import pandas as pd
import random as rnd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_squared_error

# Modules to import for Torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Check for GPU device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Loading the Data

In [3]:
# Function for min-max scaling of a DataFrame and returning min-max values
def minmax_df_with_minmax_values(df):
    result = pd.DataFrame(index=df.index, columns=df.columns)
    minmax_values = np.zeros((df.shape[1], 2), dtype=np.float32)  # Array to store min and max values
    
    for i, column in enumerate(df.columns):
        Max = df[column].max()
        Min = df[column].min()
        minmax_values[i, 0] = Min
        minmax_values[i, 1] = Max
        result[column] = (df[column] - Min) / (Max - Min)

        
    return result, minmax_values

# Function to normalize with given min-max values
def normalize_with_minmax(arr, minmax_values):
    result = np.empty_like(arr, dtype=np.float32)

    if arr.ndim == 1:
        Min = minmax_values[0]
        Max = minmax_values[1]
        for i in range(len(arr)):
            result[i] = (arr[i] - Min) / (Max - Min)
    elif arr.ndim == 2:
        for i in range(arr.shape[1]):
            Min = minmax_values[i, 0]
            Max = minmax_values[i, 1]
            for j in range(arr.shape[0]):
                result[j, i] = (arr[j, i] - Min) / (Max - Min)
        
    return result

# Example usage
array_1d = np.array([1, 2, 3, 4, 5])
minmax_values_1d = [1, 5]
normalized_1d = normalize_with_minmax(array_1d, minmax_values_1d)
print("Normalized 1D Array:\n", normalized_1d)

array_2d = np.array([[1, 2], [3, 4], [5, 6]])
minmax_values_2d = np.array([[1, 3], [6, 9]])
normalized_2d = normalize_with_minmax(array_2d, minmax_values_2d)
print("Normalized 2D Array:\n", normalized_2d)



# Example usage
data = {'A': [1, 2, 3, 4, 5], 'B': [5, 4, 3, 2, 1]}
df = pd.DataFrame(data)
scaled_df, minmax_values = minmax_df_with_minmax_values(df)

# Creating the DataFrame separately
normalized_dataframe = scaled_df.copy()
print("Scaled DataFrame:\n", normalized_dataframe)

# Display the min-max values separately
print("Min-Max Values:\n", minmax_values)


# Function for inverse min-max scaling
def inverse_transform(normalized_value, Min, Max):
    original_value = normalized_value * (Max - Min) + Min
    return original_value

# Example usage
normalized_value = 0.5
Min = 1
Max = 5
original_value = inverse_transform(normalized_value, Min, Max)
print("Original Value:", original_value)


Normalized 1D Array:
 [0.   0.25 0.5  0.75 1.  ]
Normalized 2D Array:
 [[ 0.        -1.3333334]
 [ 1.        -0.6666667]
 [ 2.         0.       ]]
Scaled DataFrame:
       A     B
0  0.00  1.00
1  0.25  0.75
2  0.50  0.50
3  0.75  0.25
4  1.00  0.00
Min-Max Values:
 [[1. 5.]
 [1. 5.]]
Original Value: 3.0


In [4]:
# Loading the data into the pandas dataframe

# Reading the Data
data = pd.read_csv("Design_Data.csv", header=None, dtype = "float32")

print("\n---------------------------------------------------------------------------\n")

# Printing the datafram 
print("\n Data loaded into pandas : \n", data)

print("\n---------------------------------------------------------------------------\n")


# # Initialize the MinMaxScaler
# scaler = MinMaxScaler()

# # Normalize the dataset
# normalized_data = scaler.fit_transform(data)

normalized_data, minmax_values = minmax_df_with_minmax_values(data)
print("\n---------------------------------------------------------------------------\n")

print("\n After Normalisation: \n ", normalized_data)
print("Min/Max Array : ",minmax_values ) 

print("\n---------------------------------------------------------------------------\n")


# # Inverse transform the normalized data
# inversed_data = scaler.inverse_transform(normalized_data)

# # Calculate RMSE to check the accuracy of the inverse transformation
# rmse = np.sqrt(mean_squared_error(data, inversed_data))

# print(f'RMSE of the inverse transformation: {rmse}')


# Inserting the data columns 
normalized_data.columns = [
    'Flapping Frequency', 'Airspeed', 'Angle Of Attack', 'Normalised Time',
    'Lift', 'Induced Drag', 'Pitching Moment', 'main wing root chord',
    'main wing wingspan', 'main wing tip chord', 'tail backward position',
    'tail root chord', 'tail tip chord', 'tail wingspan'
]

# Split the data into features and targets
X = normalized_data[[
    'Flapping Frequency', 'Airspeed', 'Angle Of Attack', 'Normalised Time',
    'main wing root chord', 'main wing wingspan', 'main wing tip chord',
    'tail backward position', 'tail root chord', 'tail tip chord', 'tail wingspan'
]]
y = normalized_data[['Lift', 'Induced Drag', 'Pitching Moment']]



minmax_values_targets = minmax_values[4:7][:]
print("target min max values : ",minmax_values_targets)

indices_to_remove = [4, 5, 6]
minmax_values_features = np.delete(minmax_values, indices_to_remove, axis=0)

print("minmax values for features :",minmax_values_features )
print("\n---------------------------------------------------------------------------\n")
print("\n Features : \n",X)
print("\n---------------------------------------------------------------------------\n")
print("\n Target : \n",y)


---------------------------------------------------------------------------


 Data loaded into pandas : 
         0    1     2         3          4         5          6    7    8   \
0      0.1  1.0 -30.0  0.000000  13.921392  1.855566   0.294453  0.3  1.4   
1      0.1  1.0 -30.0  0.500000  -9.041235 -4.439285  22.045515  0.3  1.4   
2      0.1  1.0 -25.0  0.000000  13.925155  1.909448   1.711641  0.3  1.4   
3      0.1  1.0 -25.0  0.500000 -11.013954 -4.369710  20.869232  0.3  1.4   
4      0.1  1.0 -20.0  0.000000  13.789185  1.963994   3.143793  0.3  1.4   
...    ...  ...   ...       ...        ...       ...        ...  ...  ...   
95734  0.5  5.0   5.0  0.915254  -1.671113  0.224382   0.471550  0.3  1.4   
95735  0.5  5.0   5.0  0.932203  -0.796854  0.534728   0.370605  0.3  1.4   
95736  0.5  5.0   5.0  0.949153   0.061979  0.848633   0.258699  0.3  1.4   
95737  0.5  5.0   5.0  0.966102   0.910946  1.153301   0.148894  0.3  1.4   
95738  0.5  5.0   5.0  0.983051   1.726310  1

# Splitting the data into training and testing 

In [5]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verify the split
print("Training features shape:", X_train.shape)
print("Testing features shape:", X_test.shape)
print("Training target shape:", y_train.shape)
print("Testing target shape:", y_test.shape)


Training features shape: (76591, 11)
Testing features shape: (19148, 11)
Training target shape: (76591, 3)
Testing target shape: (19148, 3)


# Defining the Model

In [6]:
class DeepNN(nn.Module):
    def __init__(self):
        super(DeepNN, self).__init__()
        self.fc1 = nn.Linear(11, 128)  # Change input features to 11
        self.dropout1 = nn.Dropout(0.5)  # Dropout layer
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.5)  # Dropout layer
        self.fc3 = nn.Linear(64, 32)
        self.dropout3 = nn.Dropout(0.5)  # Dropout layer
        self.fc4 = nn.Linear(32, 3)    # Change output features to 3
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.relu(self.fc3(x))
        x = self.dropout3(x)
        x = self.fc4(x)
        return x


In [7]:
# Convert to 2D PyTorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train.values, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test.values, dtype=torch.float32).to(device)

In [8]:
print(y_test)

tensor([[0.0140, 0.0110, 0.0095],
        [0.0139, 0.0110, 0.0095],
        [0.0142, 0.0109, 0.0095],
        ...,
        [0.0143, 0.0109, 0.0094],
        [0.0132, 0.0110, 0.0095],
        [0.0141, 0.0111, 0.0095]], device='cuda:0')


# Training and eval of the model 

In [9]:
# Move the model to GPU if available
model = DeepNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Early stopping
patience = 100
best_loss = float('inf')
best_model_state = None
no_improvement_epochs = 0

# Training the model
num_epochs = 100
train_losses = []
test_losses = []

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    train_loss = criterion(outputs, y_train)
    train_loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test)
        test_loss = criterion(test_outputs, y_test)

    train_losses.append(train_loss.item())
    test_losses.append(test_loss.item())

    if test_loss < best_loss:
        best_loss = test_loss
        best_model_state = model.state_dict()
        no_improvement_epochs = 0
    else:
        no_improvement_epochs += 1

    if no_improvement_epochs >= patience:
        print(f'Early stopping at epoch {epoch+1}')
        break

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss.item()}, Test Loss: {test_loss.item()}')

# Load the best model state
if best_model_state is not None:
    model.load_state_dict(best_model_state)


Epoch [10/100], Train Loss: 0.003279711352661252, Test Loss: 0.0004827555676456541
Epoch [20/100], Train Loss: 0.0018053946550935507, Test Loss: 0.0008029434247873724
Epoch [30/100], Train Loss: 0.0012334571219980717, Test Loss: 0.00039476784877479076
Epoch [40/100], Train Loss: 0.0009062863537110388, Test Loss: 0.0002991626097355038
Epoch [50/100], Train Loss: 0.0007135363994166255, Test Loss: 0.00021479236602317542
Epoch [60/100], Train Loss: 0.0005972040817141533, Test Loss: 0.0001704796013655141
Epoch [70/100], Train Loss: 0.0005117948167026043, Test Loss: 0.00014155646204017103
Epoch [80/100], Train Loss: 0.00044368754606693983, Test Loss: 0.00011992974032182246
Epoch [90/100], Train Loss: 0.0003917451831512153, Test Loss: 0.00010527161066420376
Epoch [100/100], Train Loss: 0.0003474343102425337, Test Loss: 9.308793232776225e-05


# testig the data from ptera software


In [10]:
import numpy as np 
import pandas as pd
from Mark4 import simulation, Mk4SaveDataToCSV
from SelfFunctions import extract_second_cycle

In [11]:
import random

def generate_random_decimal(start, end):
    return random.uniform(start, end)

# Example usage
start_value = 1.5
end_value = 5.5
random_number = generate_random_decimal(start_value, end_value)
print("Random number:", random_number)


Random number: 4.48531614247416


In [12]:
Air_Speed = generate_random_decimal(2, 6)
Angles_of_Attack = generate_random_decimal(-35, 35)
FlappingPeriod = generate_random_decimal(0.5, 1)
Normalisedtime =generate_random_decimal(0,1)
mw_root_chord = generate_random_decimal(0.3, 0.1)
mw_wingspan = generate_random_decimal(0.5, 2)
mw_tip_chord = generate_random_decimal(0.1,0.5)
tail_bposition = generate_random_decimal(0.45, 1.2)
tail_root_chord = generate_random_decimal(0.3,0.7)
tail_tip_chord = generate_random_decimal(0.001,0.1)
tail_wingspan = generate_random_decimal(0.5, 1)

print(f"Air Speed: {Air_Speed}\n"
      f"Angles of Attack: {Angles_of_Attack}\n"
      f"Flapping Period: {FlappingPeriod}\n"
      f"Main Wing Root Chord: {mw_root_chord}\n"
      f"Main Wing Wingspan: {mw_wingspan}\n"
      f"Main Wing Tip Chord: {mw_tip_chord}\n"
      f"Tail Base Position: {tail_bposition}\n"
      f"Tail Root Chord: {tail_root_chord}\n"
      f"Tail Tip Chord: {tail_tip_chord}\n"
      f"Tail Wingspan: {tail_wingspan}")


Air Speed: 4.7725268767003275
Angles of Attack: 20.535330462762992
Flapping Period: 0.581253571190786
Main Wing Root Chord: 0.2612293111494302
Main Wing Wingspan: 0.664746500891388
Main Wing Tip Chord: 0.3139811314052912
Tail Base Position: 1.193934841091074
Tail Root Chord: 0.49957619467944925
Tail Tip Chord: 0.04653467657633876
Tail Wingspan: 0.8876630354241484


In [13]:
lift, pitchingmoment, inducedsideforces = simulation(va= Air_Speed, aoa = Angles_of_Attack, fp = FlappingPeriod, mw_airfoil = "naca8304", mw_root_chord = mw_root_chord, mw_wingspan = mw_wingspan, mw_tip_chord = mw_tip_chord, tail_bposition = tail_bposition, tail_type = "V-Tail", tail_root_chord = tail_root_chord , tail_tip_chord = tail_tip_chord, tail_wingspan = tail_wingspan, tail_airfoil = "naca0004" )
print(lift, pitchingmoment, inducedsideforces)

# Flatten the arrays
lift = lift.flatten()
induced_drag = pitchingmoment.flatten()
pitching_moment = inducedsideforces.flatten()
print("Before Second Cycle Extraction :  ", lift.shape, induced_drag.shape, pitching_moment.shape)
print("Before Second Cycle Extraction value:  ", lift, induced_drag, pitching_moment)

# Extract the second cycle from the data
lift, induced_drag, pitching_moment = extract_second_cycle(lift, induced_drag, pitching_moment)
print("After Second Cycle Extraction :  ",lift.shape, induced_drag.shape, pitching_moment.shape)
print("After Second Cycle Extraction value:  ", lift, induced_drag, pitching_moment)

file_name = "test.csv"
Mk4SaveDataToCSV(FlappingPeriod, Air_Speed, Angles_of_Attack, lift, pitching_moment, induced_drag, file_name, 
     mw_root_chord, mw_wingspan, mw_tip_chord, tail_bposition, tail_root_chord, tail_tip_chord, tail_wingspan)

Simulating:100% |█████████████████████████████████████████████████| Elapsed: 00:35, Remaining: 00:00


[[ 31.93545263  -4.17087237  -2.21394658  -3.121076    -3.77082271
   -4.25436494  -4.62605318  -4.92154564  -5.16587814  -5.37758064
   -5.57095586  -5.75740558  -5.94620774  -6.14496514  -6.35977403
   -6.59521125  -6.85413909  -7.13739215  -7.44345588  -7.76836257
   -8.10590738  -8.44822144  -8.78650173  -9.11170736  -9.41505101
   -9.68838178  -9.92455598 -10.1177673  -10.26393023 -10.3608633
  -10.40868832 -10.40960362 -10.36756051 -10.28809092 -10.17780339
  -10.0438184   -9.89322953  -9.73259479  -9.56752457  -9.40240334
   -9.24025349  -9.08274008  -8.93029338  -8.7823206   -8.63746726
   -8.49389297  -8.34953362  -8.20232848  -8.05039996  -7.89218432
   -7.72651813  -7.55268974  -7.37046717  -7.18011292  -6.98239406
   -6.77858906  -6.57049255  -6.36041174  -6.15114553  -5.94594325
   -5.74834212  -5.56233009  -5.39184476  -5.24082114  -5.11301139
   -5.01187897  -4.94051073  -4.9015698   -4.89729941  -4.92949949
   -4.99951231  -5.1081812   -5.25573886  -5.44170785  -5.66473

In [15]:


# Input array
input_array = [Air_Speed, Angles_of_Attack, FlappingPeriod, 0.023809523809523808, mw_root_chord, mw_wingspan, mw_tip_chord, tail_bposition, tail_root_chord, tail_tip_chord, tail_wingspan]
input_array_normalised = normalize_with_minmax(input_array, minmax_values_features)
print("input_array_normalised", input_array_normalised)
normalised_inputarray = normalize_with_minmax()
# Apply Min-Max scaling
min_max_scaler = preprocessing.MinMaxScaler()
input_array_reshaped = [[x] for x in input_array]  # Reshape input array for scaling
x_scaled = min_max_scaler.fit_transform(input_array_reshaped)
data = pd.DataFrame(x_scaled)

# Convert to PyTorch tensor
sample_input = torch.tensor(data.values, dtype=torch.float32)

# Flatten the input tensor to match model's expected input shape
sample_input = sample_input.view(1, -1)

# Send the tensor to the device (CPU or GPU)
sample_input = sample_input.to(device)

# Print the tensor
print("Sample Input Tensor:", sample_input)

# Make predictions
with torch.no_grad():
    predictions = model(sample_input)
    print(predictions)

predictions_numpy = predictions.cpu().numpy()

print("hello =",inverse_transform(predictions_numpy[0], minmax_values[0][0], minmax_values[0][1]))

AttributeError: 'list' object has no attribute 'ndim'